<a href="https://colab.research.google.com/github/Sarnapa/Credit-Card-Fraud-Detection/blob/main/Notebooks/CreditCardFraudDetection_AnomalyDetectionAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---- Załadowanie danych


In [ ]:
# Zainstalowanie niezbędnych bibliotek
try:
    import imblearn
except ImportError:
    !pip install -q imbalanced-learn

try:
    import numpy as np
except ImportError:
    !pip install -q numpy

try:
    import pandas as pd
except ImportError:
    !pip install -q pandas

try:
    import sklearn
except ImportError:
    !pip install -q scikit-learn

# Wstępne przetwarzanie danych

## Charakterystyka danych

W naszym projekcie wykorzystujemy zbiór [Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).

Poniżej przedstawiamy podgląd zbioru *creditcardfraud*

In [ ]:

df = pd.read_csv('creditcard.csv')

print(df.head(3))


   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  
1  0.125895 -0.008983  0.014724    2.69      0  
2 -0.139097 -0.055353 -0.059752  378.66      0  

[3 rows x 31 columns]


Poniżej przedstawiamy krótką charakterystykę danych.
Warto zauważyć, że zbiór jest wysoce niezbalansowany.

In [ ]:
print("Liczba atrybutów:", df.shape[1] - 1)
print("\nCałkowita liczba transakcji:", df.shape[0])
print("Stosunek oszustw do transakcji regularnych: (0 - transakcja oznaczona jako oszustwo)\n",
      df['Class'].value_counts(normalize=True))
print("Całkowita liczba oszustw/transakcji regularnych:\n", df['Class'].value_counts())


Liczba atrybutów: 30
Całkowita liczba transakcji: 284807
Stosunek oszustw do transakcji regularnych: (0 - transakcja oznaczona jako oszustwo)
 Class
0    0.998273
1    0.001727
Name: proportion, dtype: float64
Całkowita liczba oszustw/transakcji regularnych:
 Class
0    284315
1       492
Name: count, dtype: int64


# Under / oversampling

https://www.kaggle.com/code/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets

1. Nie testować na danych, które zostały under lub oversamplowane.
2. Tutaj radzą, że jak używamy k krotną walidację to podczas jej robić to under / oversamplowanie

Plan:
1. Wpleść w k krotną walidację alternatywe ze mozna uzyc undersampling i oversampling
2. Dla undersamplingu i oversamplingu pokazac liczbe przykladow tych dwoch klas i porownanie z tym jak bylo wczesniej





In [ ]:
from enum import Enum
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

class SamplingMethod(Enum):
    NONE = "none"
    UNDER = "under"
    OVER = "over"

def preprocess_with_sampling(
    data: pd.DataFrame,
    use_cv: bool = True,
    k: int = 5,
    sampling_method: SamplingMethod = SamplingMethod.OVER,
    classifier=None
):
    """
    Funkcja przetwarzająca dane i wykonująca klasyfikację z opcjonalnym samplingiem i k-krotną walidacją.

    Parametry:
    - data: pd.DataFrame – zbiór danych z Kaggle (Credit Card Fraud Detection)
    - use_cv: bool – czy zastosować k-krotną walidację
    - k: int – liczba foldów dla k-krotnej walidacji
    - sampling_method: SamplingMethod – typ samplingu (OVER, UNDER, NONE)
    - classifier: model zgodny z API sklearn (domyślnie RandomForest)
    """
    X = data.drop(columns=["Class"])
    y = data["Class"]

    if classifier is None:
        classifier = RandomForestClassifier(random_state=42)

    if sampling_method == SamplingMethod.UNDER:
        sampler = RandomUnderSampler(random_state=42)
    elif sampling_method == SamplingMethod.OVER:
        sampler = SMOTE(random_state=42)
    else:
        sampler = None

    if use_cv:
        skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
        fold = 1
        for train_idx, test_idx in skf.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            if sampler:
                X_train, y_train = sampler.fit_resample(X_train, y_train)

            clf = classifier.fit(X_train, y_train)
            y_pred = clf.predict(X_test)

            print(f"\nFold {fold}")
            print(confusion_matrix(y_test, y_pred))
            print(classification_report(y_test, y_pred, zero_division=0))
            fold += 1
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, stratify=y, random_state=42
        )
        if sampler:
            X_train, y_train = sampler.fit_resample(X_train, y_train)

        clf = classifier.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        print(confusion_matrix(y_test, y_pred))
        print(classification_report(y_test, y_pred, zero_division=0))

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# K krotna walidacja

# Miary jakości

# Metody nadzorowane

# Metody nienadzorowane

In [ ]:

import pandas as pd

df = pd.read_csv('creditcard.csv')

print(df)


            Time         V1         V2        V3        V4        V5  \
0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
...          ...        ...        ...       ...       ...       ...   
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V21       V22  \
0       0.462388  0.239599  0.098698  0.363787  ... -0.01830